In [5]:
# !pip install transformers
# ! pip install flash-attn
# !pip install 'accelerate>=0.26.0'
# ! pip install bitsandbytes

In [14]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq, AutoTokenizer
from transformers.image_utils import load_image

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load images
image1 = load_image("https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg")
image2 = load_image("https://huggingface.co/spaces/merve/chameleon-7b/resolve/main/bee.jpg")


In [15]:
type(image1)

PIL.Image.Image

In [4]:
model_name = "HuggingFaceTB/SmolVLM-Instruct"

In [7]:
local_mode_ckpt = "/data/students/gerry/repos/jetson-vlm/local_model"

In [5]:
processor = AutoProcessor.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some kwargs in processor config are unused and will not have any effect: image_seq_len. 


In [12]:
model = AutoModelForVision2Seq.from_pretrained(local_mode_ckpt,torch_dtype=torch.bfloat16, _attn_implementation="flash_attention_2")
model = model.to(DEVICE)

In [10]:
# Initialize processor and model
# model = AutoModelForVision2Seq.from_pretrained(
#     model_name,
#     torch_dtype=torch.bfloat16,
#     _attn_implementation="flash_attention_2" if DEVICE == "cuda" else "eager",
# ).to(DEVICE)

In [11]:
save_directory = "/data/students/gerry/repos/jetson-vlm/local_model"

# Save the tokenizer and model locally
# tokenizer.save_pretrained(save_directory)
# model.save_pretrained(save_directory)

In [6]:
# from transformers import AutoModelForVision2Seq, BitsAndBytesConfig
# import torch

# quantization_config = BitsAndBytesConfig(load_in_8bit=True)
# model = AutoModelForVision2Seq.from_pretrained(
#     "HuggingFaceTB/SmolVLM-Instruct",
#     quantization_config=quantization_config,
# )

`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [13]:


# Create input messages
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "image"},
            {"type": "text", "text": "Can you describe the two images?"}
        ]
    },
]

# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1, image2], return_tensors="pt")
inputs = inputs.to(DEVICE)

# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

print(generated_texts[0])

User:<image>Can you describe the two images?
Assistant: The first image shows a statue of the Statue of Liberty in front of a cityscape with a body of water and a clear sky. The second image shows a bee on a pink flower surrounded by other flowers and plants.
